## Libraries

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate, Activation, Concatenate
from tensorflow.keras.metrics import IoU, BinaryIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
import cv2 as cv
import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19

## Loading Data

In [5]:
#home = os.environ['HOME']

In [6]:
#path_X = os.path.join(home,'raw_data/image_slices')
#path_y = os.path.join(home,'raw_data/mask_slices')

In [7]:
path_X = '/mnt/d/PDE/AerialImageDataset/slices_whole/train_image_slices'
path_y = '/mnt/d/PDE/AerialImageDataset/slices_whole/train_mask_slices'

In [8]:
split_ratio = 0.9

In [9]:
def train_val_split (path_X, path_y, split_ratio):
    X_names = os.listdir(path_X)
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    X_path = [f'{path_X}/{file}' for file in X_names]
    train_X, val_X = X_path[:int(len(X_path)*split_ratio)], X_path[int(len(X_path)*split_ratio):]
    train_y, val_y = y_path[:int(len(y_path)*split_ratio)], y_path[int(len(y_path)*split_ratio):]
    return train_X, val_X, train_y, val_y 

In [10]:
train_X, val_X, train_y, val_y = train_val_split (path_X, path_y, split_ratio)

In [11]:
def verify_matching_input_labels(X_names, y_names):
    for x, y in zip(X_names, y_names):
        if os.path.basename(x) != os.path.basename(y):
            raise ValueError(f"X and Y not matching: {x, y}")

In [12]:
verify_matching_input_labels(train_X, train_y)

In [13]:
verify_matching_input_labels(val_X, val_y)

In [14]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1) / 255 
    return image, mask

In [15]:
def batch_data (X_path, y_path, batch_size):
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.shuffle(buffer_size = len(X_path), seed = 10).map(process_path).batch(batch_size)

In [16]:
def vis_cnn_feature_maps(model, image):
    """input model and image, this function will display images after go through each CNN layer"""
    layer_names = [layer.name for layer in model.layers]
    layer_outputs = [layer.output for layer in model.layers]
    feature_map_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
    image = tf.expand_dims(image, axis=0)
    feature_maps = feature_map_model.predict(image)
    for layer_name, feature_map in zip(layer_names, feature_maps):
        if len(feature_map.shape) == 4: # Number of feature images/dimensions in a feature map of a layer 
            k = feature_map.shape[-1]  
            size=feature_map.shape[2]
            row = feature_map.shape[1]

            image_belt = np.array([[0]*k*size for i in range(row)])
            for i in range(k):
                feature_image = feature_map[0, :, :, i]  #first image of the batch for channel i
                feature_image -= feature_image.mean()
                feature_image /= feature_image.std()
                feature_image *=  64
                feature_image += 128
                feature_image = np.clip(feature_image, 0, 255)
                image_belt[:,i * size : (i + 1) * size] = feature_image

            scale = 20. / k
            plt.figure( figsize=(scale * k, scale) )
            plt.title ( layer_name )
            plt.grid  (False )
            plt.imshow(image_belt, aspect='auto')

### Training Dataset

In [50]:
train_dataset = batch_data(train_X, train_y, batch_size=4)

### Validation Dataset

In [48]:
val_dataset = batch_data(val_X, val_y, batch_size=8)

### Test Dataset

In [19]:
path_X_TEST = '/mnt/d/PDE/AerialImageDataset/slices_whole/test_image_slices'
path_y_TEST = '/mnt/d/PDE/AerialImageDataset/slices_whole/test_mask_slices'

In [20]:
def batch_data_test(path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size)

In [46]:
TEST_dataset = batch_data_test(path_X_TEST, path_y_TEST, batch_size=8)

## Model Definition

In [22]:
vgg = VGG19(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [23]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [24]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters) #can be used as skip connection 
    p = MaxPooling2D((2,2))(x)
    return x, p

In [25]:
def decoder_block(inputs, skip_features, num_filters): #skip features are going to be the x returned from the encoder block
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [26]:
def dice_loss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(targets * inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [27]:
def loss_sum(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    o = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(o)

In [59]:
def build_vgg19_unet(img_height, img_width, channels):
    
    inputs = Input((img_height, img_width, channels))
    nomorlisation = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255
    vgg19 = VGG19(include_top=False, weights='imagenet', input_tensor=nomorlisation)
    #vgg19.trainable = False
    
    #Encoder - downscaling (creating features/filter)
    skip1 = vgg19.get_layer("block1_conv2").output #shape 256, filter 64
    skip2 = vgg19.get_layer("block2_conv2").output #shape 128, filter 128
    skip3 = vgg19.get_layer("block3_conv4").output #shape 64, fitler 256
    skip4 = vgg19.get_layer("block4_conv4").output #shape 32, fitler 512
    
    #Bottleneck or bridge between encoder and decoder
    b1 = vgg19.get_layer("block5_conv4").output #shape 16, fitler 512
    
    #Decoder - upscaling (reconstructing the image and giving it precise spatial location)
    decoder1 = decoder_block(b1, skip4, 512)
    decoder2 = decoder_block(decoder1, skip3, 256)
    decoder3 = decoder_block(decoder2, skip2, 128)
    decoder4 = decoder_block(decoder3, skip1, 64)
    
    #Output
    outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(decoder4)
    model = Model(inputs, outputs)
    
    iou = BinaryIoU()
    
    model.compile(optimizer='adam', loss=loss_sum, metrics=['accuracy', iou])
    
    #model.summary()
    
    return model


In [60]:
model = build_vgg19_unet(256, 256, 3)

In [30]:
logs_dir = log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = logs_dir , histogram_freq=1)
checkpoint_filepath = '../tmp/simple_unet/loss_sum_trainingset'
es = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', restore_best_weights=True)

In [31]:
for image, mask in train_dataset:
    print(image.shape)
    print(mask.shape)
    break

(8, 256, 256, 3)
(8, 256, 256, 1)


In [45]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda_2 (Lambda)              (None, 256, 256, 3)  0           ['input_5[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 256, 256, 64  1792        ['lambda_2[0][0]']               
                                )                                                                 
                                                                                            

                                                                                                  
 activation_18 (Activation)     (None, 64, 64, 256)  0           ['batch_normalization_18[0][0]'] 
                                                                                                  
 conv2d_21 (Conv2D)             (None, 64, 64, 256)  590080      ['activation_18[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 64, 64, 256)  1024       ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_19 (Activation)     (None, 64, 64, 256)  0           ['batch_normalization_19[0][0]'] 
                                                                                                  
 conv2d_tr

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs = 500, callbacks=[es, checkpoint, tb_callback], verbose=1)

Epoch 1/500
12590/12590 [==============================] - 8579s 681ms/step - loss: 0.6166 - accuracy: 0.9147 - binary_io_u_3: 0.7226 - val_loss: 0.5495 - val_accuracy: 0.8912 - val_binary_io_u_3: 0.7627
Epoch 2/500
12590/12590 [==============================] - 8570s 681ms/step - loss: 0.4800 - accuracy: 0.9356 - binary_io_u_3: 0.7817 - val_loss: 0.5631 - val_accuracy: 0.8900 - val_binary_io_u_3: 0.7589
Epoch 3/500
12590/12590 [==============================] - 8576s 681ms/step - loss: 0.4349 - accuracy: 0.9422 - binary_io_u_3: 0.8013 - val_loss: 0.4610 - val_accuracy: 0.9055 - val_binary_io_u_3: 0.7992
Epoch 4/500
12590/12590 [==============================] - 8572s 681ms/step - loss: 0.4091 - accuracy: 0.9459 - binary_io_u_3: 0.8123 - val_loss: 0.4937 - val_accuracy: 0.9014 - val_binary_io_u_3: 0.7841
Epoch 5/500
12590/12590 [==============================] - 8574s 681ms/step - loss: 0.3846 - accuracy: 0.9491 - binary_io_u_3: 0.8225 - val_loss: 0.4665 - val_accuracy: 0.9072 - val_bi

In [ ]:
prediction = model.evaluate(TEST_dataset)

In [ ]:
plt.imshow(image[0])

In [ ]:
vis_cnn_feature_maps(model,image[0])

In [ ]:
plt.imshow(mask[0], cmap='gray')

In [ ]:
plt.imshow(model.predict(tf.expand_dims(image[0],axis=0))[0],cmap='gray')

In [57]:
mask.shape

TensorShape([8, 256, 256, 1])